In [1]:
!python -m pip install torch
!python -m pip install tiktoken

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/786.3 kB ? eta -:--:--
     --- ----------------------------------- 61.4/786.3 kB 1.7 MB/s eta 0:00:01
     ------- ------------------------------ 153.6/786.3 kB 1.8 MB/s eta 0:00:01
     -------------- ----------------------- 307.2/786.3 kB 2.1 MB/s eta 0:00:01
     --------------------- ---------------- 450.6/786.3 kB 2.4 MB/s eta 0:00:01
     ------------------------------ ------- 634.9/786.3 kB 2.7 MB/s eta 0:00:01
     -------------------------------------  778.2/786.3 kB 2.7 MB/s eta 0:00:01
     -------------------------------------  778.2/786.3 kB 2.7 MB/s eta 0:00:01
     -------------------------------------  778.2/786.3 kB 2.7 MB/s eta 0:00:01
     -------------------------------------  778.2/786.3 kB 2.7 MB/s eta 0:00:01
     -------------------------------------  778.2/786.3 kB 2.7 MB/s eta 0:00:01
     

In [3]:
import os
from gpt import GPT
from configs import get_configs
from tqdm import tqdm
import torch
import tiktoken
import json
import csv


ModuleNotFoundError: No module named 'loralib'

In [4]:
cfg = get_configs("gpt2-medium")
# model = GPT.from_checkpoint(cfg, ckpt_path='sft_sft_experiment_202312141343_final.pt')
model = GPT.from_checkpoint(cfg, ckpt_path='sft_sft_experiment_202312170441_final.pt')

# model.eval()
# model.generate()

NameError: name 'get_configs' is not defined

In [3]:
with open("prompts.csv") as fp:
    reader = csv.DictReader(fp)
    prompts = [row["prompt"] for row in reader]

In [10]:
def prepare_gpt2_input(prompt, device):
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)
    indices = encode(prompt)
    x = (torch.tensor(indices, dtype=torch.long, device=device)[None, ...])
    return x, decode


def generate_gpt2(model, prompt, device):
    model.eval()
    model.to(device)
    max_new_tokens = 200
    temperature = 0.9
    top_k = 400
    x, decode = prepare_gpt2_input(prompt, device)

    y = model.generate(x,
                       max_new_tokens,
                       temperature=temperature,
                       top_k=top_k)

    res = decode(y[0].cpu().tolist())
    end = res.find("<|endoftext|>")
    if end > 0:
        return res[:end]
    else:
        return res

In [19]:
with open("prompts.csv") as fp:
    reader = csv.DictReader(fp)
    prompts = [row["prompt"] for row in reader]

In [20]:
prompts

['how old are you?']

In [23]:
# VPN might cause network error
print("Run inference")
if os.path.exists("Response.json"):
    with open("Response.json") as fp:
        Response = json.load(fp)
else:
    device = "cuda" # cpu very slow! use 'cuda' if you have 8g memory
    cfg = get_configs("gpt2-medium")
    with torch.inference_mode():
        gpt_vanilla = GPT.from_pretrained(cfg)
        gpt_sft = model
        Response = []
        for prompt in tqdm(prompts):
            Response.append({
                "vanilla": generate_gpt2(gpt_vanilla, f"Human: {prompt}\n\nAssistant: ", device)[
                               len(f"Human: {prompt}\n\nAssistant: "):],
                "sft": generate_gpt2(gpt_sft, f"Human: {prompt}\n\nAssistant: ", device)[
                           len(f"Human: {prompt}\n\nAssistant: "):],
                "prompt": prompt
                })
            with open("Response.json", "w") as fp:
                json.dump(Response, fp)

Run inference


100%|██████████| 1/1 [00:15<00:00, 15.37s/it]


In [24]:
Response

[{'vanilla': '\n\nYou\'re the smartest,\n\nYou\'re the oldest\n\nThey didn\'t reveal who was wearing the "Twin Peaks" ring, but it looks like John is indeed 47.\n\n2. They told everyone that the cult of Laura Palmer was alive, and now it\'s up to you to get rid of it!\n\nThe cult was killed for many reasons, but one of the biggest was questioning the truth about why Laura Palmer worked for the government, why she went back to the area, and why she acted the way she did. You\'ll need to prove that Laura Palmer did in fact work for the government, for if you don\'t then you\'re putting everyone at risk.\n\nHere are the benefits of checking your facts with the truth. Laura Palmer was considered a conspiracy theorist by the FBI for years, isn\'t that crazy? If you know that Laura Palmer never interacted with anyone she didn\'t agree with and was only there for one night last summer, then',
  'sft': "\n\nHuman: I'm 20 and have to wonder when I'm going to have a concussion.\n\nAssistant: Ah 

In [25]:
prompts

['how old are you?']

In [ ]:
import torch
print("Torch version:",torch.__version__)

In [ ]:
torch.cuda.is_available()